In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.0`
import $ivy.`io.delta:delta-core_2.13:2.1.0`
import $ivy.`io.delta:delta-storage:3.0.0`

Downloaded https://repo1.maven.org/maven2/io/delta/delta-storage/3.0.0/delta-storage-3.0.0.pom


import $ivy.$
import $ivy.$
import $ivy.$

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
  .getOrCreate()

import spark.implicits._

val tablePath = "../local/delta-scala"


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/30 17:42:25 WARN Utils: Your hostname, DESKTOP-CN01VS3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 17:42:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/30 17:42:25 INFO SparkContext: Running Spark version 3.5.0
25/03/30 17:42:25 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/03/30 17:42:25 INFO SparkContext: Java version 17.0.14
25/03/30 17:42:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 17:42:25 INFO ResourceUtils: ==============================================================
25/03/30 17:42:25 INFO ResourceUtils: No custom resources configured for spark.driver.
25/03/30 17:42:25 INFO ResourceUtils: ==============================================================
25/03/30 17:42:25 INFO SparkCon

import org.apache.spark.sql.SparkSession
spark: SparkSession = org.apache.spark.sql.SparkSession@12884c74
import spark.implicits._
tablePath: String = "../local/delta-scala"

In [3]:
val df = Seq(
  ("Alice", 25),
  ("Bob", 30)
).toDF("name", "age")

25/03/30 17:42:31 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/03/30 17:42:31 INFO SharedState: Warehouse path is 'file:/home/eyal/code/notebooks/spark-warehouse'.
25/03/30 17:42:31 INFO CodeGenerator: Code generated in 174.277318 ms


df: org.apache.spark.sql.package.DataFrame = [name: string, age: int]

In [4]:
df.write
  .format("delta")
  .mode("overwrite")  // or append
  .save(tablePath)

25/03/30 17:42:33 INFO DelegatingLogStore: LogStore `LogStoreAdapter(io.delta.storage.HDFSLogStore)` is used for scheme `file`
25/03/30 17:42:33 INFO DeltaLog: Creating initial snapshot without metadata, because the directory is empty
25/03/30 17:42:33 INFO InitialSnapshot: [tableId=cae407d7-f160-420e-9df8-e1c3148ba5e2] Created snapshot InitialSnapshot(path=file:/home/eyal/code/local/delta-scala/_delta_log, version=-1, metadata=Metadata(af25a023-1919-4b19-b88a-f556836aab0d,null,null,Format(parquet,Map()),null,List(),Map(),Some(1743345753367)), logSegment=LogSegment(file:/home/eyal/code/local/delta-scala/_delta_log,-1,List(),List(),None,-1), checksumOpt=None)


java.lang.NoClassDefFoundError: org/apache/spark/sql/execution/datasources/FileFormatWriter$Empty2Null